In [8]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
# suppress warnings 
import warnings
warnings.simplefilter("ignore")

In [10]:
#Importar data
FR_df=pd.read_csv("FRvideos_cc50_202101.csv")



In [ ]:
#Improtar el json y extraer la data que nos interesa
import json 

